## Table of Contents 
1. Notebook Setup
2. Clean and Prepare Datasets for Merge/Join
3. Perform Right Join from **'steam'** to **'games'**
4. Prepare Data for Export
5. Conclusion



### 1. Notebook Setup

In [52]:
# 1.0 -- Importmpackages
import pandas as pd
import numpy as np
import regex as re
import os
import math

In [53]:
# 1.1 -- Setting our directory and importing datasets
os.chdir('C:\\Users\\Bruen\\Dropbox\\My PC (DESKTOP-V41C1LD)\\Desktop\\Data Science Portfolio\\Datasets')
games = pd.read_csv('steam_games.csv')
steam = pd.read_csv('vgsales.csv')

In [54]:
# 1.2 -- Definining a quick shape function 'bigshape' for future reference
def bigshape(x, y):
    bigshape = pd.DataFrame(list(zip(x.shape, y.shape)))\
        .rename(columns= {0: 'left', 1: 'right'})\
        .rename(index= {0: 'rows', 1: 'columns'})
    display(bigshape)
print(bigshape(games, steam))

,left,right
rows,40833,16598
columns,20,11


None


In [65]:
# 1.3 -- Let's check out our two datasets
print('\\\\ games \\\\')
display(games.head())
print('\\\\ steam \\\\')
display(steam.head())

\\ games \\


,URL,Type,Title,Description,Recent_Reviews,All_Reviews,Release_Date,Developer,Publisher,Popular_Tags,Game_Details,Languages,Achievements,Genre,Game_Description,Mature_Content,Minimum_Requirements,Recommended_Requirements,Original_Price,Discount_Price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


\\ steam \\


,Rank,Title,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### 2. Clean and Prepare Datasets for Merge/Join

In [56]:
# 2.0 -- We'll be right-joining 'games' into 'steam' along the Title column
# This is done to prioritize sales data in 'steam'
# But first, we'll reformat column names manually (sans regex)
games.columns = ['URL', 'Type', 'Title', 'Description', 'Recent_Reviews', 'All_Reviews',
       'Release_Date', 'Developer', 'Publisher', 'Popular_Tags',
       'Game_Details', 'Languages', 'Achievements', 'Genre',
       'Game_Description', 'Mature_Content', 'Minimum_Requirements',
       'Recommended_Requirements', 'Original_Price', 'Discount_Price']
steam.columns = ['Rank', 'Title', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

In [57]:
# 2.1 -- games.Title.values contains symbols and non-English characters (see examples here)
# These can be problematic for joins, especially since they are not found in steam.Titles.values
print(games.iloc[13,2])
print(games.iloc[22,2])
print(games.iloc[21433,2])

Call of Duty®: Modern Warfare® Remastered
Team Sonic Racing™
灵魂筹码 Soul at Stake


In [58]:
# 2.2 -- For example: the registered trademark symbol ®
# '®' is found in 'games'
games.Title[games.Title.str.contains('®', na= False, )]

13               Call of Duty®: Modern Warfare® Remastered
20                                Call of Duty®: Black Ops
47                     Dishonored®: Death of the Outsider™
62                            The Elder Scrolls®: Legends™
73                                        Castle Crashers®
                               ...                        
40776    Rocksmith® 2014 Edition – Remastered – Cyndi L...
40817    Rocksmith® 2014 Edition – Remastered – Cyndi L...
40827    Rocksmith® 2014 Edition – Remastered – Sabaton...
40828    Rocksmith® 2014 Edition – Remastered – Sabaton...
40829    Rocksmith® 2014 Edition – Remastered – Stone T...
Name: Title, Length: 1497, dtype: object

In [59]:
# '®' is not found in 'steam'
steam.Title[steam.Title.str.contains('®', na= False, )]

Series([], Name: Title, dtype: object)

In [60]:
# Moreover, for comparison:
steam.Title[steam.Title.str.contains('Call of Duty', na= False, )].head()

29    Call of Duty: Modern Warfare 3
31           Call of Duty: Black Ops
33         Call of Duty: Black Ops 3
34        Call of Duty: Black Ops II
35        Call of Duty: Black Ops II
Name: Title, dtype: object

In [61]:
# 2.3 -- There are currently this many titles shared between 'games' and 'steam'
print('titles shared between datasets: '+ str(steam.Title[steam.Title.isin(games.Title)].count())) # 858
print('percentage of total:            ' + str(((steam.Title[steam.Title.isin(games.Title)].count()) / len(steam))*100)) # 5.2%

titles shared between datasets: 858
percentage of total:            5.169297505723581


In [15]:
# 2.4 -- Begin standardizing df.Title.values
# Regex to remove special characters exclusive to games.Titles.values
games['Title'] = games['Title'].apply(lambda x: re.sub(r"[^a-zA-Z0-9À-ÿ:\-'\s\!\.\/\?\(\)\&\,°#\*;+·@~¡α\[\]\$♪]", '', str(x)))
steam['Title'] = steam['Title'].apply(lambda x: re.sub(r"[^a-zA-Z0-9À-ÿ:\-'\s\!\.\/\?\(\)\&\,°#\*;+·@~¡α\[\]\$♪]", '', str(x)))

In [16]:
# 2.5 Stripping white space from string ends
games['Title'] = games.Title.str.strip()
steam['Title'] = steam.Title.str.strip()

In [17]:
# All set with regex
print(games.iloc[13,2])
print(games.iloc[22,2])
print(games.iloc[21433,2])

Call of Duty: Modern Warfare Remastered
Team Sonic Racing
Soul at Stake


In [18]:
# 2.6 -- Further standardization of strings with titlecase
games['Title'] = games.Title.str.title()
steam['Title'] = steam.Title.str.title()

In [19]:
# 2.7 -- After running those numbers again:
print('titles shared between datasets: '+ str(steam.Title[steam.Title.isin(games.Title)].count())) # 1388 
print('percentage of total:            ' + str(((steam.Title[steam.Title.isin(games.Title)].count()) / len(steam))*100)) # 8.36%

titles shared between datasets: 1388
percentage of total:            8.362453307627424


In [20]:
# 2.8 -- Next, standardizing games.Release_Date & steam.Year values in YYYY format
# The current variety of values in games.Release_Date needs some attention
# Just a few examples:
games.iloc[[20, 9636, 36363, 37363, 38605, 38902, 39190, 39446, 40156, 40527], games.columns.get_loc('Release_Date')]

20                         Nov 9, 2010
9636                               NaN
36363                   When it's done
37363                      Coming soon
38605                              TBA
38902                      COMING 2019
39190                              TBD
39446    TBA - Join our Community Hub!
40156                         Sep 2019
40527                       Early 2019
Name: Release_Date, dtype: object

In [21]:
# 2.9 -- We can extract the end-of-string YYYY values with regex
# Then assign the results to a new column: game.Year
games['Release_Year'] = games.Release_Date.str.extract(r".+(\d{4}$)", expand= True)

In [22]:
# Looks like that did the trick
games.iloc[[20, 9636, 36363, 37363, 38605, 38902, 39190, 39446, 40156, 40527], games.columns.get_loc('Release_Year')]

20       2010
9636      NaN
36363     NaN
37363     NaN
38605     NaN
38902    2019
39190     NaN
39446     NaN
40156    2019
40527    2019
Name: Release_Year, dtype: object

### 3. Perform Right Join from **'steam'** to **'games'**

In [23]:
# 3.0 -- Dropping columns in 'games' that won't be needed in the join
games.drop(columns= ['Type', 'Recent_Reviews', 'Achievements', 'Minimum_Requirements', 'Recommended_Requirements'], inplace = True)

In [24]:
# 3.1 -- Perform right inner join from gamesfilt to 'steam'
# (Because we want the sales data in the steam dataset)
biggames = pd.merge(left= games, right= steam, how= 'right', left_on= 'Title', right_on= 'Title')

In [25]:
# 3.1 -- # The high NaN count in columns from 'games' is a testament to the overlap of the datasets
# i.e. not an incredibly strong overlap
def nanpct(x) :
    biggamesnan = x.isna().sum()
    biggamesnanpct = round(x.isna().sum() / len(x) * 100, 1)
    display(pd.DataFrame(pd.concat((biggamesnan, biggamesnanpct), axis = 1).rename(columns= {0: 'NaNs', 1: '#total'})))

nanpct(biggames)

,NaNs,#total
URL,15210,91.6
Title,0,0.0
Description,15213,91.6
All_Reviews,15236,91.8
Release_Date,15226,91.7
Developer,15222,91.7
Publisher_x,15216,91.6
Popular_Tags,15218,91.7
Game_Details,15211,91.6
Languages,15210,91.6


In [26]:
# 3.2 -- Next, we'll address missing Year values in biggames
# These NaNs originated in 'steam'
# Note: Numbers do not include 'Unknown' in Publisher column. More on that later.
steam.isna().sum()

Rank              0
Title             0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [27]:
# 3.3 -- Fillna to compile YYYYs in biggames.Year
biggames.Year.fillna(biggames.Release_Year, inplace= True)

In [28]:
# This corrected 33 values in our merged dataset
steam.Year.isna().sum() - biggames.Year.isna().sum()

33

In [29]:
# 3.4 -- Our next goal: use Publisher_x to correct problem values in Publisher_y
# Just a few samples:
biggames.iloc[[4648, 5626, 8845, 9798, 14410, 16559], \
              [1,biggames.columns.get_loc('Publisher_x'), biggames.columns.get_loc('Publisher_y')]]

,Title,Publisher_x,Publisher_y
4648,Castlevania: Curse Of Darkness,NaN,Konami Digital Entertainment
5626,Singularity,"Activision,Activision",Unknown
8845,Terraria,"Re-Logic,Re-Logic",Unknown
9798,Ecco The Dolphin,"SEGA,SEGA",Sega
14410,Take On Helicopters,"Bohemia Interactive,Bohemia Interactive",Unknown
16559,Bound By Flame,"Focus Home Interactive,Focus Home Interactive",NaN


In [30]:
# 3.5 -- First, replace 'Unknown' with np.NaN
biggames.Publisher_y.replace('Unknown', np.NaN, inplace= True)

In [31]:
# 3.66 -- Next, fill NaNs in Publisher_y with values from Publisher_x
biggames.Publisher_y.fillna(biggames.Publisher_x, inplace= True)

In [32]:
# 3.7 -- Removing duplicates from our newly-formatted Publisher-y column
# We can use split, expand it, and select the first column with .get(0) 
# Then we'll reassign it to Publisher_y
biggames['Publisher_y'] = biggames.Publisher_y.str.split(',', expand = True).get(0)

In [33]:
# Samples are looking much better
biggames.iloc[[4648, 5626, 8845, 9798, 14410, 16559], \
              [1,biggames.columns.get_loc('Publisher_x'), biggames.columns.get_loc('Publisher_y')]]

,Title,Publisher_x,Publisher_y
4648,Castlevania: Curse Of Darkness,NaN,Konami Digital Entertainment
5626,Singularity,"Activision,Activision",Activision
8845,Terraria,"Re-Logic,Re-Logic",Re-Logic
9798,Ecco The Dolphin,"SEGA,SEGA",Sega
14410,Take On Helicopters,"Bohemia Interactive,Bohemia Interactive",Bohemia Interactive
16559,Bound By Flame,"Focus Home Interactive,Focus Home Interactive",Focus Home Interactive


In [34]:
# 3.8 -- Checkpoint: what's the status of our NaNs?
nanpct(biggames)

,NaNs,#total
URL,15210,91.6
Title,0,0.0
Description,15213,91.6
All_Reviews,15236,91.8
Release_Date,15226,91.7
Developer,15222,91.7
Publisher_x,15216,91.6
Popular_Tags,15218,91.7
Game_Details,15211,91.6
Languages,15210,91.6


In [35]:
# 3.9 -- Again, not an incredibly strong overlap, but remember:
# Our original column was full of "Unknown" values, as well as NaNs
print('original # of Unknowns: ' + str(len(steam[steam['Publisher'] == 'Unknown'])))
print('original # of NaN       ' + str(len(steam[steam['Publisher'].isna()])))
print('total:                = ' + str((len(steam[steam['Publisher'].isna()])) + \
                                        len(steam[steam['Publisher'] == 'Unknown'])))
print('current # of Unknowns:  ' + str(len(biggames[biggames['Publisher_y'] == 'Unknown'])))
print('current # of NaNs:      ' + str(len(biggames[biggames['Publisher_y'].isna()])))
print('total:                = ' + str((len(biggames[biggames['Publisher_y'].isna()])) + \
                                        len(biggames[biggames['Publisher_y'] == 'Unknown'])))

original # of Unknowns: 203
original # of NaN       58
total:                = 261
current # of Unknowns:  0
current # of NaNs:      236
total:                = 236


### 4. Prepare Data for Export

In [36]:
# 4.0 -- First, drop redundant and irrelevant columns
biggames.drop(columns = ['Release_Date', 'Release_Year', 'Publisher_x', 'Genre_x'], inplace= True)
biggames.drop(columns = ['URL', 'Description', 'All_Reviews', 'Developer',
       'Popular_Tags', 'Game_Details', 'Languages', 'Game_Description',
       'Mature_Content', 'Original_Price', 'Discount_Price'], inplace= True)


In [37]:
# 4.1 -- And now, rename our remaining columns
biggames.columns = ['Title', 'Rank', 'Platform', 'Year', 'Genre', 'Publisher',
       'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

In [38]:
# 4.2 Reorder columns for readability
biggames = biggames[['Title', 'Rank', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]

In [39]:
# 4.3 Rename game platforms for colloqial familiarity
biggames.replace({'Platform' : {
    'Wii' : 'Nintendo Wii',
    'NES' : 'Nintendo NES',
    'GB' : 'Game Boy', 
    'DS' : 'Nintendo DS',
    'X360' : 'Xbox 360', 
    'GEN' : 'Sega Genesis',
    'PS3' : 'PlayStation 3',
    'PS2' : 'PlayStation 2',
    'SNES' : 'Super Nintendo',
    'GBA' : 'Game Boy Advance',
    '3DS' : 'Nintendo 3DS',
    'PS4' : 'PlayStation 4',
    'N64': 'Nintendo 64',
    'PS' : 'PlayStation',
    'XB' : 'Xbox',
    '2600' : 'Atari 2600',
    'PSP' : 'Sony PSP',
    'XOne' : 'Xbox One',
    'GC' : 'GameCube',
    'WiiU' : 'Nintendo WiiU',
    'DC' : 'Dreamcast',
    'PSV' : 'PlayStation Vita',
    'SAT' : 'Sega Saturn',
    'SCD' : 'Sega CD',
    'WS' : 'WonderSwan',
    'NG' : 'Neo Geo',
    'TG16' : 'TurboGrafx-16',
    'GG' : 'Game Gear',
    'PCFX' : 'PC-FX'},
    'Genre' : {'Platform' : 'Platformer'} # And also correct this Genre value
}, inplace = True)

In [40]:
# 4.4 Use melt to reshape Region/Sales values 
biggames = biggames.melt(['Title', 'Rank', 'Platform', 'Year', 'Genre', 'Publisher'], var_name= 'Region', value_name= 'Sales')

### 5. Conclusion

In [42]:
# 5.0 Here's a look at our final 'biggames' dataframe
biggames.head()

,Title,Rank,Platform,Year,Genre,Publisher,Region,Sales
0,Wii Sports,1,Nintendo Wii,2006.0,Sports,Nintendo,NA_Sales,41.49
1,Super Mario Bros.,2,Nintendo NES,1985.0,Platformer,Nintendo,NA_Sales,29.08
2,Mario Kart Wii,3,Nintendo Wii,2008.0,Racing,Nintendo,NA_Sales,15.85
3,Wii Sports Resort,4,Nintendo Wii,2009.0,Sports,Nintendo,NA_Sales,15.75
4,Pokemon Red/Pokemon Blue,5,Game Boy,1996.0,Role-Playing,Nintendo,NA_Sales,11.27


In [66]:
# 5.1 -- Using our custom function to compare NaNs
print('\\\\ games \\\\')
display(nanpct(games))
print('\\\\ steam \\\\')
display(nanpct(steam))
print('\\\\ biggames \\\\')
display(nanpct(biggames))

\\ games \\


,NaNs,#total
URL,0,0.0
Type,2,0.0
Title,16,0.0
Description,13221,32.4
Recent_Reviews,38127,93.4
All_Reviews,12363,30.3
Release_Date,3179,7.8
Developer,343,0.8
Publisher,5100,12.5
Popular_Tags,2945,7.2


None

\\ steam \\


,NaNs,#total
Rank,0,0.0
Title,0,0.0
Platform,0,0.0
Year,271,1.6
Genre,0,0.0
Publisher,58,0.3
NA_Sales,0,0.0
EU_Sales,0,0.0
JP_Sales,0,0.0
Other_Sales,0,0.0


None

\\ biggames \\


,NaNs,#total
Title,0,0.0
Rank,0,0.0
Platform,0,0.0
Year,1190,1.4
Genre,0,0.0
Publisher,1180,1.4
Region,0,0.0
Sales,0,0.0


None

In [44]:
# 5.2 And one last shape measurement
bigshape(biggames, steam)

,left,right
rows,83020,16598
columns,8,11


In [ ]:
# 5.3 -- The biggames dataframe is now ready for export.
biggames.to_csv(path_or_buf= 'C:\\Users\\Bruen\\Dropbox\\My PC (DESKTOP-V41C1LD)\\Desktop\\TopGlobalGameSalesMelt.csv')

### Main takeaways from this project thus far:

1. Both 'games' and 'steam' were adequately-sized datasets
2. 'games' was over twice the size of 'steam', but had much less integrity (NaN %, Title formatting)
3. 'games' proved to be very inconsistent - only 36 NaNs in one column, over 37000 NaNs in another column


Personal thoughts:

1. 'steam' was pleasant to work with, and was prioritized for its sales data
2. To avoid disturbing the accuracy of global rankings, I decided against dropping biggames titles with NaN Publisher values
2. To allow for more creativity/flexibility, I hope to work with larger, more consistent datasets in the future
3. Looking forward to plotting this out in Tableau!